In [ ]:
from agent import TelosAgent
from environment import TelosTaskEnv, make_env
from task import TelosTask
from standing_task.standing_task import StandingTelosTask
from standing_task.standing_environment import StandingTelosTaskEnv,make_standing_env
from time import sleep
import pybullet as p
from utils.PyBullet import PyBullet
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3 import SAC, PPO
import numpy as np
from stable_baselines3.common.sb2_compat.rmsprop_tf_like import RMSpropTFLike

from stable_baselines3.common.env_checker import check_env

In [2]:
pb = PyBullet(render_mode="rgb_array",renderer="Tiny")
telos_agent = TelosAgent(pb)
telos_task = StandingTelosTask(agent=telos_agent,sim_engine=pb)
telos_env = StandingTelosTaskEnv(task=telos_task, agent=telos_agent, sim_engine=pb)

In [ ]:
telos_env.reset()
telos_env.step(telos_env.action_space.sample())

In [9]:
goal = telos_env.step(telos_env.action_space.sample())[4]["goal_position"]

In [10]:
obs = telos_env.step(telos_env.action_space.sample())[0]["agent"]
obs = obs[7:31]
joint_pos,joint_vel = obs[::2],obs[1::2]
centre_of_mass = pb.get_center_of_mass(1)

In [ ]:
f_standing,A_standing = rmp_standing(joint_pos,joint_vel,goal,centre_of_mass)
f_joint_limits,A_joint_limits = RMP_joint_limits(joint_pos,joint_vel, )

In [ ]:
f_standing,A_standing

In [4]:
env = make_vec_env(make_standing_env, n_envs=8, env_kwargs={"task": telos_task, "agent": telos_agent, "sim": pb})
env = VecNormalize(env, norm_obs=True, norm_reward=True)

In [5]:
policy_sac =  dict(net_arch=dict(pi=[512, 512, 512], qf=[512, 512,512]))

In [6]:
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.05 * np.ones(n_actions))

In [ ]:
model_sac = SAC(
    policy="MultiInputPolicy",
    env=env,
    policy_kwargs=policy_sac,
    # action_noise= action_noise,
    learning_rate=1e-3,
    buffer_size= int(1e5),
    batch_size= 2048,
    ent_coef='auto',
    gamma= 0.997,
    tau= 0.005,
    # train_freq= 1,
    # gradient_steps= -1,
    # optimize_memory_usage= False,
    # target_update_interval= 1,
    verbose= 1,
    tensorboard_log="./tensorboard/standing_task/sacTelos/",
)

In [ ]:
model_sac.learn(1_000_000)

In [ ]:
model_sac.save("sac_standing_task_3")

In [ ]:
pb_t = PyBullet(render_mode="rgb_array",renderer="OpenGL")
telos_agent_t = TelosAgent(pb_t)
telos_task_t = StandingTelosTask(agent=telos_agent_t,sim_engine=pb_t)
telos_env_t = StandingTelosTaskEnv(task=telos_task_t, agent=telos_agent_t, sim_engine=pb_t)

In [ ]:
test_model = SAC.load("sac_standing_task")